In [0]:
# Imports

import pandas as pd
import numpy as np
import re
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.model_selection import train_test_split


In [29]:
# Import the data files to the local directory


!wget <LINK> -O temp.zip; unzip temp.zip; rm temp.zip


/bin/bash: LINK: No such file or directory
unzip:  cannot find or open temp.zip, temp.zip.zip or temp.zip.ZIP.
rm: cannot remove 'temp.zip': No such file or directory


In [0]:
# Extract the textual information from the .csv to a dataframe

df = pd.read_csv(r'/content/DetectNow/1586829496995_68613e22-7df3-11ea-8ed6-062008888c7e/export.csv')
df.head()

,age,consent,corona_test,filename,gender,medical_history,patientID,patient_id,patient_reported_symptoms,sampleType,data_t,smoker,video_patient_finder_tip_filename
0,41 - 55,yes,Negative,https://detectnowdatalake.blob.core.windows.ne...,male,"Diabetes with complications,",DMC-202,637,"Fever, chills, or sweating,New or worsening co...",study,NaN,NaN,NaN
1,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,female,Disease or conditions that make it harder to c...,20030511135111357902,648,"New or worsening cough,Sore throat,",study,NaN,NaN,NaN
2,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,NaN,200413111464227,649,"Fever, chills, or sweating,",study,NaN,NaN,NaN
3,18 - 30,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,NaN,200413111464200,650,NaN,guest,NaN,NaN,NaN
4,31 - 40,yes,Not Tested,https://detectnowdatalake.blob.core.windows.ne...,male,NaN,200413111463989,652,"New or worsening cough,Loss of taste,Loss of s...",study,NaN,NaN,NaN


In [0]:
# Extract Medical History and convert to dictionary with numbers

med_history_array = np.asarray(df['medical_history'])



med_history_array = str(med_history_array)


## Parse the med_history_array and find unique comma seperated strings 

med_history_dict = {}
index = 0

for entry in med_history_array.splitlines():
    history_list = entry.split(",")
    for hist in history_list:
      hist = hist.replace('\'' , '')
      hist = hist.replace('[' , '')
      hist = hist.replace(']' , '')
      hist = hist.strip()
      if hist not in med_history_dict and 'nan' not in hist and hist is not '':    
        med_history_dict[hist] = index       
        index += 1


print("The dictionary with numbers: ", med_history_dict)


The dictionary with numbers:  {'Diabetes with complications': 0, 'Disease or conditions that make it harder to cough': 1, 'Asthma or chronic lung disease': 2, 'Congestive heart failure': 3, 'None': 4, 'Extreme obesity': 5}


In [0]:
# Assign the patients numerical history combination to an array

x_hist_text_train = np.zeros(len(df['medical_history']))

index = 0

for entry in df['medical_history']:
  if type(entry) is not float:
    num_seq = ''
  
    for string in entry.split(","):
      if string is not '':
        num_seq += str(med_history_dict[string])


    x_hist_text_train[index] = float(num_seq)
    
  else: 
    x_hist_text_train[index] = -1.0
  index += 1


print('The shape of the x_text_train is ' , x_hist_text_train.shape)


The shape of the x_text_train is  (37,)


In [0]:
# Extract patient reported symptoms and convert to dictionary with numbers 

symptom_array = np.asarray(df['patient_reported_symptoms'])

symptom_array = str(symptom_array)


## Parse the med_history_array and find unique comma seperated strings 

symptom_dict = {}
index = 0

for entry in symptom_array.splitlines():
    split_entry = re.findall('[A-Z][^A-Z]*', entry)
    for symptom in split_entry:
      symptom = symptom.strip().replace(',' , '')
      symptom = symptom.strip().replace('\'' , '')
      symptom = symptom.replace('nan' , '').strip()
      symptom = symptom.replace(']' , '')
      if symptom not in symptom_dict:
        symptom_dict[symptom] = index
        index += 1

print("The dictionary with numbers: ", symptom_dict)


The dictionary with numbers:  {'Fever chills or sweating': 0, 'New or worsening cough': 1, 'Sore throat': 2, 'Loss of taste': 3, 'Loss of smell': 4, 'Shortness of breath': 5, 'Body aches': 6}


In [0]:
# Assign the patients numerical symptom combination to an array

x_symp_text_train = np.zeros(len(df['medical_history']))

index = 0

for entry in df['patient_reported_symptoms']:
  if type(entry) is not float:
    num_seq = ''
    split_entry = re.findall('[A-Z][^A-Z]*', entry)

    for symptom in split_entry:
        symptom = symptom.strip().replace(',' , '')
        symptom = symptom.strip().replace('\'' , '')
        symptom = symptom.replace('nan' , '').strip()
        symptom = symptom.replace(']' , '')

        num_seq += str(symptom_dict[symptom])
    x_symp_text_train[index] = float(num_seq)
      
    
  else: 
    x_hist_text_train[index] = -1.0
  index += 1


print('The shape of the x_text_train is ' , x_symp_text_train.shape)


The shape of the x_text_train is  (37,)


In [0]:
# Convert the y strings to digits

df.loc[df['corona_test'] == 'Negative', 'corona_test'] = 0
df.loc[df['corona_test'] == 'Positive', 'corona_test'] = 1
df.loc[df['corona_test'] == 'Not Tested', 'corona_test'] = -1

y_text_array = np.asarray(df['corona_test'])




In [24]:
print('The x medical history sequence array is: ' , x_hist_text_train)
print('The x symptoms sequence array is: ' , x_symp_text_train)
print('The shape of the y_text_train is ' , y_text_array.shape)



The x medical history sequence array is:  [ 0.  1. -1. -1. -1. -1. -1.  2. -1. -1. -1.  1.  1.  1. -1.  3. -1. -1.
  1. -1. -1.  2. -1. -1. -1. -1.  2. -1. -1. -1. -1. 23. -1. -1.  2.  4.
  5.]
The x symptoms sequence array is:  [1.000e+00 1.200e+01 0.000e+00 0.000e+00 1.340e+02 5.340e+02 1.000e+00
 5.134e+03 0.000e+00 1.000e+00 5.100e+01 2.000e+00 1.000e+00 1.000e+00
 0.000e+00 1.260e+02 0.000e+00 0.000e+00 5.160e+02 0.000e+00 0.000e+00
 5.100e+01 1.200e+01 5.100e+01 0.000e+00 5.000e+00 2.600e+01 5.100e+01
 1.200e+01 2.000e+00 5.120e+02 5.000e+00 5.263e+03 0.000e+00 5.000e+00
 5.000e+00 0.000e+00]
The shape of the y_text_train is  (37,)


In [0]:
# Split into train, validate, and test

## Question: How should we split the data-sets?


X_train, X_test, y_train, y_test = train_test_split(x_hist_text_train, y_text_array, test_size=0.2, random_state=1) # What shouldd the X be?

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

## Another question: How should we combine the medical history and the symptoms?

In [0]:
# Random Forest Classifer

def rf_model(x_train , y_train , num_branches):

  model = rf()
  model.fit(x_train , y_train)
  
  # Evaluate the model
  


In [0]:
#  - SVM 

In [0]:
# - Decision Tree

In [0]:
# TODO: Build more complex models

# - Deep Learning Model ???
# Things to try

# - Fully connected NN
# - K means clustering on the digit vectors from the text
